### Nonlinear Option Pricing (MATHGR5400), Columbia University, Spring 2020
# Homework III

### Due Date: 11:55 PM Thursday, April 2, 2020
You should turn in the notebook at Columbia CourseWorks website

Please comment your code properly.

Before you turn in the notebook, press the "Run all cells" button in the toolbar, and make sure all the calculation results and graphs are produced correctly in a reasonable time frame, and then save the notebook.

# Uncertain Mortality Model vs American Options

** Re-insurance Deals**

In this assignment, we study the *Uncertain Mortality Model* for the pricing of reinsurance deals. For the sake of simplicity, in this assignment we only consider one type of default: the risk of death.

We consider the following reinsurance product:

- At maturity, if the insurance subscriber is alive, the issuer delivers a put on the underlying $X$

$$
u^\textrm{mat}( x) = (K_\textrm{mat} - x)_+.
$$

- At the time of death, if it is before the maturity, the issuer delivers an exit payoff, typically another put on the underlying $X$.

$$
u^D(t, x) = (K_D - x)_+.
$$

- The subscriber pays a constant fee $α\Delta t$ at every time step until death or the maturity or the product

- The insurance sells a large number of these contracts to subscribers. We assume that the times of death of the subscribers $\tau^D$ are independent, and identically distributed, and also independent of the underlying's stock price.

- We assume that the underlying's risk neutral price dynamics is the Black-Scholes model with zero interest rate/repo/dividend yield

$$
dX_t = \sigma X_t d W_t.
$$

** The Insurers' Approach and Risk-Neutral Pricing**

This contract shows two types of risk: the times of death of the subscribers and the changes in the price of the underlying. 

In this case, the issuer can apply the insurer's approach to the risk of death times, i.e., the law of large numbers. The more people buy the contract, the less risk.

Choosing a risk-neutral measure under which the death times $\tau^D$ have the same distribution as under the historical probability measure is equivalent to applying the arbitrage-pricing approach to the financial risk insurer's rule on the risk of death. The price of the contract is then

$$
u(t, x) = \mathbb{E}^\mathbb{Q}_{t, x} \left[u^\textrm{mat}(X_T) \mathbb{1}_{\tau^D \geq T} -\int_t^T\alpha\mathbb{1}_{\tau^D>s}ds+ u^D(\tau^D, X_{\tau^D}) \mathbb{1}_{\tau^D < T} \right].
$$

** Deterministic Death Rate **

If the death intensity is a deterministic function $\lambda_t^D$ (i.e. $\tau^D$ has an exponential distribution with time-dependent intensity $\lambda_t^D$), then we have seen that $u$ is the solution to the linear PDE

$$
\left\{\begin{array}{l}
\partial_t u + \frac{1}{2} \sigma^2 x^2 \partial_x^2 u -\alpha + \lambda_t^D \cdot (u^D - u) = 0,\\
u(T, \cdot) \equiv u^\textrm{mat}.
\end{array}\right.
$$

** Uncertain Mortality Model **

If the death rate is uncertain, we assume that it is adapted (i.e., it does not look into the future) and belongs to a moving corridor
$\left[\underline{\lambda}_t, \overline{\lambda}_t\right]$. The most conservative way to price the contract is to compute the (financially) worst death rate process $\lambda_t^D$ as being chosen so as to maximize the value of the contract. The resulting HJB equation is

$$
\left\{\begin{array}{l}
\partial_t u + \frac{1}{2} \sigma^2 x^2 \partial^2_x u -\alpha + \Lambda^D(t, u^D - u) \cdot (u^D - u) = 0,\\
u(T, \cdot) \equiv u^\textrm{mat},
\end{array}\right.
$$
where the function $\Lambda^D$ is defined by
$$
\Lambda^D (t, y) = \left\{\begin{array}{l}
    \overline{\lambda}^D_t \quad \textrm{if} \ y \geq 0, \\
    \underline{\lambda}^D_t \quad \textrm{otherwise.}
\end{array}\right.
$$

## Link with $1$-BSDE and Numerical Schemes

From the Pardoux-Peng theorem, we know that the solution $u(0, x)$ can be represented as the solution $Y_0^x$ to the $1$-BSDE

$$
dY_t = -f(t, X_t, Y_t, Z_t) \, dt + Z_t \, dW_t,
$$

with the terminal condition $Y_T = u^\textrm{mat} (X_T)$, where $X_0 = x$ and

$$
f(t, x, y, z) = -\alpha + \Lambda^D(t, u^D(t, x) - y) \cdot (u^D(t, x) - y).
$$


## BSDE Discretization

### Explicit Euler Schemes

\begin{align*}
Y_{t_n} =& \ u^{\text{mat}}\left(X_{t_n}\right)\\
Y_{t_{i - 1}} =& \ \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] -\alpha\Delta t_i + \Lambda^D \left(t_{i - 1}, u^D(t_{i - 1} , X_{t_{i - 1}}) - \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ]\right) \cdot \left( u^D(t_{i - 1}, X_{t_{i - 1}}) - \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] \right) \Delta t_i\\
=& -\alpha\Delta t_i + \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] \left( 1 - \overline{\lambda}^D \Delta t_i \right) +
u^D(t_{i - 1}, X_{t_{i - 1}}) \overline{\lambda}^D \Delta t_i \right) \mathbb{1}_{u^D\left(t_{i - 1}, X_{t_{i - 1}}\right) \geq \mathbb{E}_{i-1}[Y_{t_i}]}\\
& + \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] \left( 1 - \underline{\lambda}^D \Delta t_i \right) +
u^D(t_{i - 1}, X_{t_{i - 1}}) \underline{\lambda}^D \Delta t_i \right) \mathbb{1}_{u^D\left(t_{i - 1}, X_{t_{i - 1}}\right) < \mathbb{E}_{i-1}[Y_{t_i}]}.
\end{align*}

### Implicit Euler Scheme

\begin{align*}
Y_{t_n} =& \ u^{\text{mat}}\left(X_{t_n}\right)\\
Y_{t_{i - 1}} =& \ \mathbb{E}^\mathbb{Q}_{i - 1} [Y_{t_i}] -\alpha\Delta t_i+ \Lambda^D \left(t_{i - 1}, u^D(t_{i - 1}, X_{t_{i - 1}}) - Y_{t_{i - 1}}\right) \cdot \left( u^D(t_{i - 1}, X_{t_{i - 1}}) - Y_{t_{i - 1}} \right) \Delta t_i.
\end{align*}

The implicit scheme involves $Y_{t_{i - 1}}$ on both sides of the equation, which generally requires a root-finding routine to find $Y_{t_{i - 1}}$. However in this specific case, it can be shown that 

\begin{equation}u^D(t_{i-1}, X_{t_{i-1}}) \geq Y_{t_{i-1}}\quad\text{if and only if}\quad u^D(t_{i-1}, X_{t_{i-1}}) \geq \mathbb{E}^{\mathbb{Q}}_{i-1}\left[Y_{t_i}\right]-\alpha\Delta t_i.\end{equation}

Thus

\begin{split}
Y_{t_{i - 1}} =& \frac{1}{1 + \overline{\lambda}^D \Delta t_i} \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] - \alpha\Delta t_i +
u^D(t_{i - 1}, X_{t_{i - 1}}) \overline{\lambda}^D \Delta t_i \right) \mathbb{1}_{u^D\left(t_{i - 1}, X_{t_{i - 1}}\right) \geq \mathbb{E}_{i-1}[Y_{t_i}]-\alpha\Delta t_i}\\
& + \frac{1}{1 + \underline{\lambda}^D \Delta t_i} \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] -\alpha\Delta t_i +
u^D(t_{i - 1}, X_{t_{i - 1}}) \underline{\lambda}^D \Delta t_i \right) \mathbb{1}_{u^D(t_{i - 1}, X_{t_{i - 1}}) < \mathbb{E}_{i-1}[Y_{t_i}]-\alpha\Delta t_i}.
\end{split}



In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

In [2]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    """Generate Monte-Carlo paths in Black-Scholes model.

    Parameters
    ----------
    S: scalar
        The spot price of the underlying security.
    vol: scalar
        The implied Black-Scholes volatility.
    r: scalar
        The annualized risk-free interest rate, continuously compounded.
    q: scalar
        The annualized continuous dividend yield.
    ts: array_like
        The time steps of the simualtion
    npaths: int
        the number of paths to simulate

    Returns
    -------
    paths: ndarray
        The Monte-Carlo paths.
    """
    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=np.float),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths

def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    """Compute the call/put option price in the Black-Scholes model
    
    Parameters
    ----------
    K: scalar or array_like
        The strike of the option.
    T: scalar or array_like
        The maturity of the option, expressed in years (e.g. 0.25 for 3-month and 2 for 2 years)
    S: scalar or array_like
        The current price of the underlying asset.
    vol: scalar or array_like
        The implied Black-Scholes volatility.
    r: scalar or array_like
        The annualized risk-free interest rate, continuously compounded.
    q: scalar or array_like
        The annualized continuous dividend yield.
    callput: str
        Must be either 'call' or 'put'.

    Returns
    -------
    price: scalar or array_like
        The price of the option.

    Examples
    --------
    >>> blackscholes_price(95, 0.25, 100, 0.2, r=0.05, callput='put')
    1.5342604771222823
    """
    F = S*np.exp((r-q)*T)
    v = np.sqrt(vol**2*T)
    d1 = np.log(F/K)/v + 0.5*v
    d2 = d1 - v
    try:
        opttype = {'call':1, 'put':-1}[callput.lower()]
    except:
        raise ValueError('The value of callput must be either "call" or "put".')
    price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
    return price


def pwlinear_basis(xknots):
    """Basis that represent a piecewise linear function with given knots"""
    fs = [lambda x: np.ones_like(x, dtype=np.float), lambda x: x-xknots[0]]
    fs.extend([lambda x, a=xknots[i]: np.maximum(x-a, 0) for i in range(len(xknots))])
    return fs


def pwlinear_fit(xdata, ydata, xknots):
    """Fit a piecewise linear function with xknots to xdata and ydata"""
    fs = pwlinear_basis(xknots)
    A = np.vstack([f(xdata) for f in fs]).T
    ps = np.linalg.lstsq(A, ydata, rcond=None)[0]
    return ps, fs

<b>Note.</b> To be more specific about the payouts of the product, we make the following assumptions:

- We assume that all transactions occur at the end of the month, i.e. the insurance subsriber is expected to pay a fee of $\alpha/12$ at the end of each month; if the subscriber dies in the middle of the month, the death payment will be made at the end of that month. 

- If the death occurs at the maturity, we assume that the death comes first and the death payment $u^D$ will be made.

## Assignment III

<b style="color:darkorange">Question 1</b> (Deterministic Mortality Rate Model)

We assume that the underlying's risk neutral price dynamics is the Black-Scholes model with zero interest rate/repo/dividend yield

$$dX_t = \sigma X_t d W_t,\quad X_0=100,\quad\sigma=0.3$$

We shall price the reinsurance deal using two Monte Carlo methods explained below.

<b>(a).</b> (Direct simulation of death times) Implement a Monte Carlo simulation by directly simulating death time $\tau^D$ for each path to estimate the quantity

$$u(t, x) = \mathbb{E}^\mathbb{Q}_{t, x} \left[u^\textrm{mat}(X_T) \mathbb{1}_{\tau^D \geq T} - \int_t^T\alpha \mathbb{1}_{\tau^D> s}ds
+ u^D(\tau^D, X_{\tau^D}) \mathbb{1}_{\tau^D < T} \right],$$

when $T=10$, $K_{\text{mat}}=90$, $K_D=100$, $\alpha=3$, and $\tau^D$ has an exponential distribution of constant intensity $\lambda^D=0.025$. Use monthly time steps.


<b>(b).</b> (Averaging over death events) Using the Feynman-Kac formula, derive a stochastic representation of the linear PDE 

$$\left\{\begin{array}{l}\partial_t u + \frac{1}{2}\sigma^2 x^2 \partial_x^2 u - \alpha + \lambda^D \cdot (u^D - u) = 0\\
        u(T, x) \equiv u^\textrm{mat}(x)\end{array}\right.$$
        
that does not involve simulating death times $\tau^D$. Explain how one can get this stochastic representation from the one given in (a). Implement the corresponding Monte Carlo simulation to estimate $u(0,X_0)$.


__Answers__

<b>(a).</b> (Direct simulation of death times)

In [3]:
# note that lambda is a reserved keyword in python, so we use lambd instead
def exponential_samples(n, lambd):
    """Generate n samples from exponential distribution with rate lambd"""
    """Inverse Transform: X~exp(lambda), then cdf of X -- F(X) has uniform distr (use F^-1(U) to sample X)"""
    return -np.log(np.random.rand(n))/lambd

In [4]:
X0, vol, r, q= 100, 0.3, 0, 0
K_mat, K_D = 90, 100
alpha, lambd = 3, 0.025
ts = np.linspace(0, 10, 121) #10 year, monthly payment, 0~10 with 121 numbers
n_paths = 100000
paths = blackscholes_mc(X0, vol, r, q, ts, n_paths)
t_death = exponential_samples(n_paths, lambd)  #death distribution

In [5]:
tot_payoff = 0
for i in range(1,len(ts)):
    idx = t_death<=(ts[i])  #appear death within month
    #no discount, r=0
    death_payoff = np.maximum(K_D-paths[i,idx],0)   # death payment
    tot_payoff += np.sum(death_payoff-alpha*t_death[idx])  #deduct fee (alpha*t_death here for death)
    paths = paths[:,~idx]
    t_death = t_death[~idx]
    if i==len(ts)-1:
        mat_payoff = np.maximum(K_mat-paths[i,:],0) #maturity payment
        tot_payoff += np.sum(mat_payoff-alpha*ts[-1]) #deduct fee (alpha*T here for maturity, can not use t_death)

V = tot_payoff/n_paths        
print("Direct simulation price: ", V)

Direct simulation price:  2.087534140761238


<b>(b).</b> (Averaging over death events)

Stochastic representation (b):
$$
u(t,x) = \mathbb{E}^\mathbb{Q}_{t,x}\left[e^{-\int_{t}^{T}\lambda^D ds} u^{mat}(X_T)\right] + \mathbb E^\mathbb{Q}_{t,x}\left[\int_{t}^{T}e^{-\int_{t}^{s}\lambda^D dv} (-\alpha + u^{D}(s,X_s)\lambda^D)ds\right]
$$

Explanation: we can derive (b) from (a) by taking in  $\mathbb{1}_{\tau^D \geq T}=e^{-\int_{t}^{T}\lambda^D ds}, \mathbb{P}_{\tau^D \in (s,s+ds) |\tau\geq t} = \lambda^D e^{-\int_{t}^{s}\lambda^D dv} $

\begin{align}
(a) &= \mathbb{E}^\mathbb{Q}_{t, x} \left[u^\textrm{mat}(X_T) \mathbb{1}_{\tau^D \geq T} - \int_t^T\alpha \mathbb{1}_{\tau^D> s}ds
+ u^D(\tau^D, X_{\tau^D}) \mathbb{1}_{\tau^D < T} \right] \\
 &= \mathbb{E}^\mathbb{Q}_{t,x}\left[e^{-\int_{t}^{T}\lambda^D ds} u^{mat}(X_T)\right] + \mathbb{E}^\mathbb{Q}_{t, x}\left[\int_{t}^{T}-\alpha e^{-\int_{t}^{s}\lambda^D dv}+\int_{t}^{T}u^{D}(s,X_s)(\lambda^D e^{-\int_{t}^{s}\lambda^D dv}) ds\right] = (b)
\end{align}

Represented by numerical schemes:

- At $T$,  $C_{T} = u^{mat}(t_n,X_{T})$
- At $0\leq t_k<T$, $C_{t_k} = \exp\left(-\lambda^D \Delta_{t_{k+1}}\right) C_{t_{k+1}} - \alpha*\Delta_{t_{k+1}} + \left(1-\exp(-\lambda^D \Delta_{t_{k+1}})\right) u^D(t_{k+1},X_{t_{k+1}})$
- Average all $u_{t_0}$ across N paths to estimate the fair value of the contract at time zero

In [6]:
X0, vol, r, q= 100, 0.3, 0, 0
K_mat, K_D = 90, 100
alpha, lambd = 3, 0.025
ts = np.linspace(0, 10, 121) #10 year, monthly payment, 0~10 with 121 numbers
n_paths = 100000
paths = blackscholes_mc(X0, vol, r, q, ts, n_paths)

In [7]:
V = np.maximum(K_mat-paths[-1,:],0)
#backward and average N paths
for i in range(len(ts)-2,0,-1):
    dt = ts[i+1]-ts[i]
    mat_payoff = np.exp(-lambd*dt)*V  #maturity payoff under measure
    payment  = -alpha*dt
    death_payoff = (1-np.exp(-lambd*dt))*np.maximum(K_D-paths[i+1,:],0)  #death payoff under measure
    V = mat_payoff + payment + death_payoff

V = np.mean(V)
print("Events average price: ", V)

Events average price:  2.1473512824321923


<b style="color:darkorange">Question 2</b> (Uncertain Mortality Rate Model - BSDE)

Assume that $\lambda^D_t \in \left[\underline{\lambda}^D, \overline{\lambda}^D \right]$, where $\underline{\lambda}^D=0.005$ and $\overline{\lambda}^D=0.04$.

<b>(a).</b> Implement the implicit Euler scheme for BSDE. Use piecewise-linear fit with 10 knots to estimate the conditional expectation $\mathbb{E}^{\mathbb{Q}}_{i-1}\left[Y_{t_i}\right]$. Use the numpy function percentile to define the minimum and maximum knots. Use the percentiles 1 and 99 percent. Use 100,000 paths.

<b>(b).</b> Run an independent new simuation. Use the estimates of $\mathbb{E}^{\mathbb{Q}}_{i-1}\left[Y_{t_i}\right]$ obtained in (a) to get a lower bound estimate.

<b>(c).</b> How does the price compare with the price in the model with deterministic mortality rate $\underline{\lambda}^D$? with deterministic mortality rate $\overline{\lambda}^D$? Check numerically.

__Answers__

<b>(a).</b> (BSDE with Implicit Euler Scheme)

\begin{split}
Y_{t_{i - 1}} =& \frac{1}{1 + \overline{\lambda}^D \Delta t_i} \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] - \alpha\Delta t_i +
u^D(t_{i - 1}, X_{t_{i - 1}}) \overline{\lambda}^D \Delta t_i \right) \mathbb{1}_{u^D\left(t_{i - 1}, X_{t_{i - 1}}\right) \geq \mathbb{E}_{i-1}[Y_{t_i}]-\alpha\Delta t_i}\\
& + \frac{1}{1 + \underline{\lambda}^D \Delta t_i} \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] -\alpha\Delta t_i +
u^D(t_{i - 1}, X_{t_{i - 1}}) \underline{\lambda}^D \Delta t_i \right) \mathbb{1}_{u^D(t_{i - 1}, X_{t_{i - 1}}) < \mathbb{E}_{i-1}[Y_{t_i}]-\alpha\Delta t_i}.
\end{split}

In [8]:
lambda_max, lambda_min = 0.04, 0.005
X0, vol, r, q= 100, 0.3, 0, 0
K_mat, K_D = 90, 100
ts = np.linspace(0, 10, 121) #10 year, monthly payment, 0~10 with 121 numbers
n_paths = 100000

In [9]:
## First simulation
paths = blackscholes_mc(X0, vol, r, q, ts, n_paths)
V = np.maximum(K_mat-paths[-1,:],0)
Ps, Fs = [_ for i in range(len(ts))], [_ for i in range(len(ts))]
for i in range(len(ts)-2,0,-1):
    dt = ts[i+1]-ts[i]
    ##use piecewise-linear fit to get cond expect
    xknots = np.linspace(np.percentile(paths[i,:], 1), np.percentile(paths[i,:], 99), 10)
    ps, fs = pwlinear_fit(paths[i,:], V, xknots)
    Ps[i], Fs[i] = ps,fs
    cond_v=sum([f(paths[i,:])*p for (f, p) in zip(fs, ps)])
    
    death_payoff = np.maximum(K_D-paths[i,:],0)    #death payoff
    idx = death_payoff >= (cond_v-alpha*dt)   #where to use max price
    V_max = 1/(1 + lambda_max*dt)*(cond_v - alpha*dt + death_payoff*lambda_max*dt)
    V_min = 1/(1 + lambda_min*dt)*(cond_v - alpha*dt + death_payoff*lambda_min*dt)
    V[idx] = V_max[idx]    #use max
    V[~idx] = V_min[~idx]  #use min
    
V = np.mean(V)
print("Implicit BSDE model price: ", V)

Implicit BSDE model price:  3.5435420847575965


__(b).__ (Independent new simuation)

Wrong way:

The below is not right, we should use the parameters estimated in (a) to estimate lambda but not used for price value directly. Otherwise we are still maintaining the regression error. Then we use this lambda to produce the guidance, i.e. to default or not, to help advice the price at each step following the formula in Q1(b). Now we are no longer using implicit schema but using the direct expectation formula in Q1 (b). Notcie like longstaff-shwartz, we are not using the regression but the guidance to estimate a lower-bound price.

Notice the wrong way below, if we use the parameters before to directly estimate the value at each step, we do not need to do the backward loop. We can only use the a single step paths\[0\] to get the value V without loop, since each step we do not use V_{i+1} to update V_{i}, but only use paths\[i\] and paramters.

In [ ]:
## Second independent simulation
new_paths = blackscholes_mc(X0, vol, r, q, ts, n_paths)
V = np.maximum(K_mat-new_paths[-1,:],0)

#below is wrong and not necessary
for i in range(len(ts)-2,0,-1):
    dt = ts[i+1]-ts[i]
    ##use piecewise-linear from (a) to get cond expect
    ps,fs = Ps[i], Fs[i]
    cond_v=sum([f(new_paths[i,:])*p for (f, p) in zip(fs, ps)])
    
    death_payoff = np.maximum(K_D-new_paths[i,:],0)    #death payoff
    idx = death_payoff >= (cond_v-alpha*dt) #where to use min price
    V_max = 1/(1 + lambda_max*dt)*(cond_v - alpha*dt + death_payoff*lambda_max*dt)
    V_min = 1/(1 + lambda_min*dt)*(cond_v - alpha*dt + death_payoff*lambda_min*dt)
    V[idx] = V_max[idx]    #use max
    V[~idx] = V_min[~idx]  #use min
    
V = np.mean(V)
print("(Wrong) Independent implicit BSDE model price: ", V)

Right way:
Use parameters in part (a) to get the estimation of lambda, and use lambda to guide the estimation following the schema in Q1 (b)

- At $T$,  $C_{T} = u^{mat}(t_n,X_{T})$
- At $0\leq t_k<T$, $C_{t_k} = \exp\left(-\lambda^D \Delta_{t_{k+1}}\right) C_{t_{k+1}} - \alpha*\Delta_{t_{k+1}} + \left(1-\exp(-\lambda^D \Delta_{t_{k+1}})\right) u^D(t_{k+1},X_{t_{k+1}})$


In [ ]:
## Second independent simulation
new_paths = blackscholes_mc(X0, vol, r, q, ts, n_paths)
V = np.maximum(K_mat-new_paths[-1,:],0)
lambd = np.zeros_like(new_paths)

# use regression parameters to decide the lambda
for i in range(len(ts)-2,0,-1):
    dt = ts[i+1]-ts[i]
    ##use piecewise-linear from (a) to get cond expect
    ps,fs = Ps[i], Fs[i]
    cond_v=sum([f(new_paths[i,:])*p for (f, p) in zip(fs, ps)])
    death_payoff = np.maximum(K_D-new_paths[i,:],0)    #death payoff
    #compare to decide where to use lambda_max or lambda_min
    lambd[i] = np.where(death_payoff >= (cond_v-alpha*dt), lambda_max, lambda_min)

V = np.maximum(K_mat-new_paths[-1],0)
#use lambda in the q1(b) formula to price the value
for i in range(len(ts)-2,0,-1):
    dt = ts[i+1]-ts[i]
    death_payoff = np.maximum(K_D-new_paths[i,:],0)    #death payoff
    V = np.exp(-lambd[i]*dt)*V - alpha*dt + (1-np.exp(-lambd[i]*dt))*death_payoff

V = np.mean(V)
print("Independent implicit BSDE model price: ", V)

__(c).__ (Compare price with fixed rate numerically)

- Using fixed $\overline\lambda_D$


\begin{split}
Y_{t_{i - 1}} =& \frac{1}{1 + \overline{\lambda}^D \Delta t_i} \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] - \alpha\Delta t_i +
u^D(t_{i - 1}, X_{t_{i - 1}}) \overline{\lambda}^D \Delta t_i \right)
\end{split}



__Q2(c)__: answer is off for both \lambdas. There is no need to do any regression for deterministic \lambda case since it will introduce unnecessary regression errors, should use methods in Q1. (-0.5)

In [11]:
## 1. First simulation
paths = blackscholes_mc(X0, vol, r, q, ts, n_paths)
V0 = np.maximum(K_mat-paths[-1,:],0)
Ps, Fs = [_ for i in range(len(ts))], [_ for i in range(len(ts))]
for i in range(len(ts)-2,0,-1):
    dt = ts[i+1]-ts[i]
    ##use piecewise-linear fit to get cond expect
    xknots = np.linspace(np.percentile(paths[i,:], 1), np.percentile(paths[i,:], 99), 10)
    ps, fs = pwlinear_fit(paths[i,:], V0, xknots)
    Ps[i], Fs[i] = ps,fs
    cond_v=sum([f(paths[i,:])*p for (f, p) in zip(fs, ps)])
    
    death_payoff = np.maximum(K_D-paths[i,:],0)    #death payoff
    V0 = 1/(1 + lambda_max*dt)*(cond_v - alpha*dt + death_payoff*lambda_max*dt)
    
## 2. Second independent simulation
new_paths = blackscholes_mc(X0, vol, r, q, ts, n_paths)
V1 = np.maximum(K_mat-new_paths[-1,:],0)
for i in range(len(ts)-2,0,-1):
    dt = ts[i+1]-ts[i]
    ##use piecewise-linear from (a) to get cond expect
    ps,fs = Ps[i], Fs[i]
    cond_v=sum([f(new_paths[i,:])*p for (f, p) in zip(fs, ps)])
    
    death_payoff = np.maximum(K_D-new_paths[i,:],0)    #death payoff
    V1 = 1/(1 + lambda_max*dt)*(cond_v - alpha*dt + death_payoff*lambda_max*dt)
    
V1 = np.mean(V1)
print("Fixed upper mortality rate model price under BSDE: ", V1)


Fixed upper mortality rate model price under BSDE:  3.486645233193959


- Using fixed $\underline\lambda_D$

\begin{split}
Y_{t_{i - 1}} =& \frac{1}{1 + \underline{\lambda}^D \Delta t_i} \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] - \alpha\Delta t_i +
u^D(t_{i - 1}, X_{t_{i - 1}}) \underline{\lambda}^D \Delta t_i \right)
\end{split}

In [12]:
## 1. First simulation
paths = blackscholes_mc(X0, vol, r, q, ts, n_paths)
V0 = np.maximum(K_mat-paths[-1,:],0)
Ps, Fs = [_ for i in range(len(ts))], [_ for i in range(len(ts))]
for i in range(len(ts)-2,0,-1):
    dt = ts[i+1]-ts[i]
    ##use piecewise-linear fit to get cond expect
    xknots = np.linspace(np.percentile(paths[i,:], 1), np.percentile(paths[i,:], 99), 10)
    ps, fs = pwlinear_fit(paths[i,:], V0, xknots)
    Ps[i], Fs[i] = ps,fs
    cond_v=sum([f(paths[i,:])*p for (f, p) in zip(fs, ps)])
    
    death_payoff = np.maximum(K_D-paths[i,:],0)    #death payoff
    V0 = 1/(1 + lambda_min*dt)*(cond_v - alpha*dt + death_payoff*lambda_min*dt)
    
## 2. Second independent simulation
new_paths = blackscholes_mc(X0, vol, r, q, ts, n_paths)
V1 = np.maximum(K_mat-new_paths[-1,:],0)
for i in range(len(ts)-2,0,-1):
    dt = ts[i+1]-ts[i]
    ##use piecewise-linear from (a) to get cond expect
    ps,fs = Ps[i], Fs[i]
    cond_v=sum([f(new_paths[i,:])*p for (f, p) in zip(fs, ps)])
    
    death_payoff = np.maximum(K_D-new_paths[i,:],0)    #death payoff
    V1 = 1/(1 + lambda_min*dt)*(cond_v - alpha*dt + death_payoff*lambda_min*dt)
    
V1 = np.mean(V1)
print("Fixed lower mortality rate model price under BSDE: ", V1)

Fixed lower mortality rate model price under BSDE:  0.41589639297603387


The price in uncertain mortality model is higher than both the price under the deterministic mortality rate $\underline{\lambda}^D$ and $\overline{\lambda}^D$. This is because the uncertain mortality rate model always take the "sup" value considering every situations while the deterministic rate model use the same lambda, which might get sub-optimal for certain situations.


<b style="color:darkorange">Question 3</b> (Uncertain Mortality Rate Model - Longstaff and Schwartz) In addition to the BSDE simulation, the reinsurance deals can be priced by adapting the Longstaff-Schwartz algorithm. An implementation of the Longstaff-Schwartz algorithm is provided below for your convenience.

The same lower and upper bounds of $\lambda_t^D$ are used as Question 2.

<b>(a).</b> In the example code, we use European put price (the strike is taken to be the average of $K_{\text{mat}}$ and $K_D$) and constant as basis functions to estimate the sum of future payoffs. Modify the code by using the piecewise-linear fit to estimate the sum of future payoffs.

<b>(b).</b> Compare the BSDE scheme with the Longstaff-Schwartz-like scheme in terms of variance. For each method run the two-step procedure (regression and indedenpent pricing) 20 times to estimate variance. Use 5000 paths for regression and 50000 for indepedent pricing.

<b>(c).</b> What is approximately the value of the fee $\alpha$ that makes the deal costless at inception?

__Answers__

<b>(a).</b> (Longstaff-Schwartz price)

In [13]:
# Longstaff-Schwartz algorithm for Uncertain Mortality Rate Model
S = 100
vol = 0.3
T = 10
ts = np.linspace(0, T, int(np.round(T*12))+1)
alpha = 3
lambda_min = 0.005
lambda_max = 0.04

K_mat = 90.0
K_D = 100.0

def u_mat(x):
    return np.maximum(K_mat - x, 0)

def u_D(x):
    return np.maximum(K_D - x, 0)


# first Monte Carlo run to estimate the value function by regressions
npaths = 5000
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
Ps, Fs = [_ for i in range(len(ts))], [_ for i in range(len(ts))]
V = u_mat(paths[-1])
lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)

for i in range(len(ts)-2, 0, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
    xknots = np.linspace(np.percentile(paths[i,:], 1), np.percentile(paths[i,:], 99), 10)
    ps, fs = pwlinear_fit(paths[i,:], V, xknots)
    Ps[i], Fs[i] = ps, fs
    cond_v = sum([f(paths[i,:])*p for (f, p) in zip(fs, ps)])
    lambd = np.where(u_D(paths[i]) >= cond_v, lambda_max, lambda_min)

# independent simulation to obtain a lower bound
npaths = 100000
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
V = u_mat(paths[-1])
lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
for i in range(len(ts)-2, 0, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
    ps, fs = Ps[i], Fs[i]
    cond_v = sum([f(paths[i,:])*p for (f, p) in zip(fs, ps)])
    lambd = np.where(u_D(paths[i]) >= cond_v, lambda_max, lambda_min)
    
print("Uncertain mortality rate model price under Longstaff-Schwartz: ", np.mean(V))

Uncertain mortality rate model price under Longstaff-Schwartz:  3.515160655320706


<b>(b).</b> (Comparing variance of BSDE and Longstaff-Schwartz)
- BSDE variance

In [14]:
npaths1, npaths2 = 5000, 50000
V_list = []
alpha = 3

for n in range(0,20):
    ## First simulation
    paths = blackscholes_mc(X0, vol, r, q, ts, npaths1)
    V = np.maximum(K_mat-paths[-1,:],0)
    Ps, Fs = [_ for i in range(len(ts))], [_ for i in range(len(ts))]
    for i in range(len(ts)-2,0,-1):
        dt = ts[i+1]-ts[i]
        ##use piecewise-linear fit to get cond expect
        xknots = np.linspace(np.percentile(paths[i,:], 1), np.percentile(paths[i,:], 99), 10)
        ps, fs = pwlinear_fit(paths[i,:], V, xknots)
        Ps[i], Fs[i] = ps,fs
        cond_v=sum([f(paths[i,:])*p for (f, p) in zip(fs, ps)])
    
        death_payoff = np.maximum(K_D-paths[i,:],0)    #death payoff
        idx = death_payoff >= (cond_v-alpha*dt)   #where to use max price
        V_max = 1/(1 + lambda_max*dt)*(cond_v - alpha*dt + death_payoff*lambda_max*dt)
        V_min = 1/(1 + lambda_min*dt)*(cond_v - alpha*dt + death_payoff*lambda_min*dt)
        V[idx] = V_max[idx]    #use max
        V[~idx] = V_min[~idx]  #use min

    ## Second independent simulation
    new_paths = blackscholes_mc(X0, vol, r, q, ts, npaths2)
    V = np.maximum(K_mat-new_paths[-1,:],0)

    for i in range(len(ts)-2,0,-1):
        dt = ts[i+1]-ts[i]
        ##use piecewise-linear from (a) to get cond expect
        ps,fs = Ps[i], Fs[i]
        cond_v=sum([f(new_paths[i,:])*p for (f, p) in zip(fs, ps)])
    
        death_payoff = np.maximum(K_D-new_paths[i,:],0)    #death payoff
        idx = death_payoff >= (cond_v-alpha*dt) #where to use min price
        V_max = 1/(1 + lambda_max*dt)*(cond_v - alpha*dt + death_payoff*lambda_max*dt)
        V_min = 1/(1 + lambda_min*dt)*(cond_v - alpha*dt + death_payoff*lambda_min*dt)
        V[idx] = V_max[idx]    #use max
        V[~idx] = V_min[~idx]  #use min
    
    V = np.mean(V)
    V_list.append(V)

print("BSDE price Variance: ",np.var(np.array(V_list)))

BSDE price Variance:  0.17581055952482627


- Longstaff-Schwartz variance

In [15]:
npaths1, npaths2 = 5000, 50000
V_list = []
alpha = 3

for n in range(0,20):
    # first Monte Carlo run to estimate the value function by regressions    
    paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths1)
    Ps, Fs = [_ for i in range(len(ts))], [_ for i in range(len(ts))]
    V = u_mat(paths[-1])
    lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)

    for i in range(len(ts)-2, 0, -1):
        dt = ts[i+1]-ts[i]
        p = 1-np.exp(-lambd*dt)
        V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
        xknots = np.linspace(np.percentile(paths[i,:], 1), np.percentile(paths[i,:], 99), 10)
        ps, fs = pwlinear_fit(paths[i,:], V, xknots)
        Ps[i], Fs[i] = ps, fs
        cond_v = sum([f(paths[i,:])*p for (f, p) in zip(fs, ps)])
        lambd = np.where(u_D(paths[i]) >= cond_v, lambda_max, lambda_min)

    # independent simulation to obtain a lower bound
    
    paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths2)
    V = u_mat(paths[-1])
    lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
    for i in range(len(ts)-2, 0, -1):
        dt = ts[i+1]-ts[i]
        p = 1-np.exp(-lambd*dt)
        V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
        ps, fs = Ps[i], Fs[i]
        cond_v = sum([f(paths[i,:])*p for (f, p) in zip(fs, ps)])
        lambd = np.where(u_D(paths[i]) >= cond_v, lambda_max, lambda_min)
    
    V = np.mean(V)
    V_list.append(V)

print("Longstaff-Schwartz price variance: ",np.var(np.array(V_list)))

Longstaff-Schwartz price variance:  0.011934856094844664


We can see that the variance of price under the longstaff-schwartz uncertain mortality model is much lower than that of the BSDE model.

<b>(c)</b> Find $\alpha$ for costless deal

Q3(c): should have a systemic way of computing \alpha*, e.g. use root finding schemes such as bisection method and set a threshold for error. (-0.25)

In [16]:
# Longstaff-Schwartz algorithm for Uncertain Mortality Rate Model
# Test different alpha
alpha_value = np.linspace(3, 3.5, 11)

for alpha in alpha_value:
    # first Monte Carlo run to estimate the value function by regressions
    npaths = 5000
    paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
    Ps, Fs = [_ for i in range(len(ts))], [_ for i in range(len(ts))]
    V = u_mat(paths[-1])
    lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)

    for i in range(len(ts)-2, 0, -1):
        dt = ts[i+1]-ts[i]
        p = 1-np.exp(-lambd*dt)
        V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
        xknots = np.linspace(np.percentile(paths[i,:], 1), np.percentile(paths[i,:], 99), 10)
        ps, fs = pwlinear_fit(paths[i,:], V, xknots)
        Ps[i], Fs[i] = ps, fs
        cond_v = sum([f(paths[i,:])*p for (f, p) in zip(fs, ps)])
        lambd = np.where(u_D(paths[i]) >= cond_v, lambda_max, lambda_min)

    # independent simulation to obtain a lower bound
    npaths = 100000
    paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
    V = u_mat(paths[-1])
    lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
    for i in range(len(ts)-2, 0, -1):
        dt = ts[i+1]-ts[i]
        p = 1-np.exp(-lambd*dt)
        V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
        ps, fs = Ps[i], Fs[i]
        cond_v = sum([f(paths[i,:])*p for (f, p) in zip(fs, ps)])
        lambd = np.where(u_D(paths[i]) >= cond_v, lambda_max, lambda_min)
    
    print("alpha:",alpha, " price:", np.mean(V))

alpha: 3.0  price: 3.3801820961843907
alpha: 3.05  price: 3.162260814406363
alpha: 3.1  price: 2.5850999610576215
alpha: 3.15  price: 2.098438800630626
alpha: 3.2  price: 1.7688009551016897
alpha: 3.25  price: 1.2814693498099017
alpha: 3.3  price: 0.8634934436191093
alpha: 3.35  price: 0.5375258618204807
alpha: 3.4  price: -0.030015881891466797
alpha: 3.45  price: -0.33329534068621625
alpha: 3.5  price: -0.7522967090622963


We can see that when the fee $\alpha$ is around 3.38, the deal is costless at the beginning.

 

<b style="color:darkorange">Question 4.</b> Set $K_D=0$, and all the other parameters remain the same, in particualr $\alpha=3$. Then the default event is equivalent to lapse (with no mortality payoff). It has been observed that insurance subscribers do not lapse optimally, which explains the use of the uncertain lapse model with minimum and maximum lapse rates $\underline{\lambda}$, $\overline{\lambda}$. 

<b>(a).</b> If insurance subscribers were to lapse optimally, i.e., exercise optimally their option to lapse, how would you price the contract?

<b>(b).</b> Explain how you can get this price (from optimal exercise) in the uncertain lapse model. Reuse the code provided in Question 3 to show your result (you may have to change some parameters in the code).

<b>(c).</b> Implement Longstaff-Schwartz algorithm to price the American option with payoff $u^D=0$ (Always perform a second indepedent run to get a clean lower bound price). Compare the price from uncertain lapse model in part (b) with the American price.

__Answers__

<b>(a).</b> (Optimal lapse contract price)
We use the same uncertain motality model with longstaff-schwartz algorithm as in problem (c), except now we set different lambda values for $\lambda$. Since the $\lambda$ function accounts for the fact that policeholders may lapse by they should not, we can get optimal lapse when setting $\underline\lambda=0$, $\overline\lambda=+\infty$. We can prove the model is now of optimal lapse as follows.

after setting $\underline\lambda=0$, $\overline\lambda=+\infty$, we have the longstaff-shwartz-like method as 
- At $T$,  $C_{T}^p = u^{mat}(t_n,X_{T})$
- At $0\leq t_k<T$, $C_{t_k}^p = \exp\left(-\lambda^{D,p} \Delta_{t_{k+1}}\right) C_{t_{k+1}} - \alpha*\Delta_{t_{k+1}} + \left(1-\exp(-\lambda^{D,p} \Delta_{t_{k+1}})\right) u^D(t_{k+1},X_{t_{k+1}})$

where 
$\lambda_{t_k}^{D,p}={+\infty}$  if $u^D(t_k,X_{t_k}^p)<\hat{u}(t_k,X_{t_k}^p)$; $\quad \lambda_{t_k}^{D,p}=0$ if $u^D(t_k,X_{t_k}^p)\geq\hat{u}(t_k,X_{t_k}^p)$, and $\hat{u}(t_k,X_{t_k}^p)$ is the regression estimate。

Set the optimal lapse time as $t_{opt}$. We know that $u^D=0$, and $u(t_k,X_{t_k}^p) = \max(u^D,\hat u(t_{k},X_{t_k}^p))$. Here we find the last optimal (lapse) exercise time from backward and ignore the difference between the first optimal lapse time and the last one. 

Before $t_{opt}$ at $t$, we have $u^D(t_k,X_{t_k}^p)< \hat u(t_k,X_{t_k}^p)$, so the price formula becomes $C_{t_k} =  C_{t_{k+1}} - \alpha*\Delta_{t_{k+1}}$, showing the contract should continue. But if at time $t_{opt}$ we have $u^D(t_k,X_{t_k}^p) \geq\hat u(t_k,X_{t_k}^p)$, the price turns to 
$C_{t_k} =  - \alpha*\Delta_{t_{k+1}} + u^D(t_{k+1},X_{t_{k+1}})$, which is exactly the lapse value, showing the contract should be lapsed immediately (at the optimal lapse time). Thus in this way, we can price the contract under the optimal lapse time.


<b>(b)</b> (Price uncertain lapse model with optimal exercise)

By setting the lower-bound lambda to 0 and upper-bound lambda to infinity, we can get the optimal lapse and derive the price using the code in question 3.

In [17]:
# Longstaff-Schwartz algorithm for Uncertain Mortality Rate Model
S, vol, T = 100, 0.3, 10
ts = np.linspace(0, T, int(np.round(T*12))+1)
alpha = 3
lambda_max, lambda_min = float("inf"), 0  #use a very large number to denote infinity
K_mat, K_D = 90.0, 0
def u_mat(x):
    return np.maximum(K_mat - x, 0)
def u_D(x):
    return np.maximum(K_D - x, 0)

# first Monte Carlo run to estimate the value function by regressions
npaths = 5000
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
betas = np.zeros((len(ts), 2), dtype=np.float)
V = u_mat(paths[-1])
lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
K = 0.5*(K_mat+K_D)
for i in range(len(ts)-2, 0, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas[i] = np.linalg.lstsq(A, V, rcond=None)[0]
    lambd = np.where(u_D(paths[i]) >= betas[i, 0]+betas[i, 1]*Z, lambda_max, lambda_min)

# independent simulation to obtain a lower bound
npaths = 100000
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
V = u_mat(paths[-1])
lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
K = 0.5*(K_mat+K_D)
for i in range(len(ts)-2, -1, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
    lambd = np.where(u_D(paths[i]) >= betas[i, 0]+betas[i, 1]*Z, lambda_max, lambda_min)

print("Optimal lapse contract price under Longstaff-Schwartz: ", np.mean(V))

Optimal lapse contract price under Longstaff-Schwartz:  3.398156747639663


<b>(c)</b> (Price an American Option with $u^D=0$)

To price the American option, we need to compare the exercise (lapse) value and the continuation value. To achieve this, we run the backward regression as before and then use forward (rather than backward as before) loop to compare each step and update the price. The final price is the average of all paths price.

__Remark Q4(c)__: Confused LS for American options and LS for reinsurance deals. There is no \lambda in LS for American option pricing. Please see the algo in HW1 for reference. (-1)

In [18]:
# Longstaff-Schwartz algorithm for American Option
S, vol, T = 100, 0.3, 10
ts = np.linspace(0, T, int(np.round(T*12))+1)
alpha = 3
lambda_max, lambda_min = float("inf"), 0

K_mat, K_D = 90.0, 0

def u_mat(x):
    return np.maximum(K_mat - x, 0)

def u_D(x):
    return np.maximum(K_D - x, 0)


# first Monte Carlo run to estimate the value function by regressions
npaths = 5000
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
betas = np.zeros((len(ts), 2), dtype=np.float)
V = u_mat(paths[-1])
lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)

K = 0.5*(K_mat+K_D)
for i in range(len(ts)-2, 0, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas[i] = np.linalg.lstsq(A, V, rcond=None)[0]
    lambd = np.where(u_D(paths[i]) >= betas[i, 0]+betas[i, 1]*Z, lambda_max, lambda_min)


# independent simulation to obtain a lower bound (from beginning to the end)
npaths = 100000
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
exer = np.zeros(npaths)
V = np.zeros(npaths)  #initial price value
for i in range(1, len(ts)):
    if i==len(ts)-1:
        exer_val = u_mat(paths[-1])
        idx = (exer==0)
        V[idx] = exer_val[idx]-alpha*ts[i]
        break
    
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
    cont_val = betas[i, 0]+betas[i, 1]*Z
    #cont_val = sum([f(paths[i])*p for (f, p) in zip(fs, ps)])
    exer_val = u_D(paths[i])
    idx = (exer_val>cont_val) & (exer==0)
    V[idx] = exer_val[idx]-alpha*ts[i]  #each time when exercised, we shall add on the payment fee in between
    exer[idx] = 1 #udpate exercise status

print("American Option price under Longstaff-Schwartz: ", np.mean(V))

American Option price under Longstaff-Schwartz:  3.4144817078232537


The price for monthly payment American Option in part (c) is almost the same (slightly higher) as the price for the price of uncertain lapse model in part (b).